# Shortlisted Tags

In [33]:
tags = {'javascript', 'java', 'c#', 'php', 'python', 'android', 'jquery', 'html', 'c++', 'ios', 'css', 'asp', 
        'R', 'C', 'rails', 'node', 'net', 'objective-c', 'json', 'sql', 'angular', 'swift', 'django', 
        'excel', 'ruby', 'ajax', 'angular', 'xml', 'asp.net', 'linux', 'react', 'spring', 'wordpress','laravel', 
        'mongodb', 'bash', 'git', 'typescript', 'bootstrap', 'scala', 'matlab', 'apache','hibernate', 
        'firebase', 'azure','shell', 'maven', 'selenium','.htaccess', 'docker', 'codeigniter', 'symfony', 'perl', 
        'cordova', 'express', 'tensorflow', 'unity', 'ubuntu', 'visual studio', 'vue', 'delphi','haskell', 'unix', 
        'go', 'hadoop', 'jpa', 'tomcat', 'cocoa', 'nginx', 'curl', 'selenium', 'laravel', 'gradle', 'flash'}

user_map = {}
d1 = {}

# Parses out any tagging done via HTML tags

In [34]:
import re

def cleanhtml(raw_html):
    
    '''
    Considering each post consists of some <HTML> tags,
    this parses out the tags
    '''
    
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [35]:
cleanhtml("I love Python<Python>")

'I love Python'

# Pickling to cache data

In [36]:
import pickle
def pickle_file(data,name):
    
    '''
    Pickle files taking data and name of the file 
    '''
    
    pkl = open(name,"wb")
    pickle.dump(data,pkl)
    pkl.close()

def unpickle_file(name):
    
    '''
    Unpickle files from name of the file 
    '''
    
    try:
        pkl = open(name,"rb")
        d1 = pickle.load(pkl)
        pkl.close()
        return d1
    except:
        return "Error"

# Jaccard Similarity calculator

In [37]:
def jaccard_similarity(question1,question2):
    
    '''
    Jaccard similarity between question1 and question2
    '''
    
    bigram1 = build_bigram(question1)
    bigram2 = build_bigram(question2)
    q1 = set(bigram1)
    q2 = set(bigram2)
    jacc = None
    try:
        jacc = float(len(q1.intersection(q2))/len(q1.union(q2)))
    except:
        jacc = 0.0
    return jacc

# Trigram is more specific, and helps get extremely unique results. However, this requires the graph to be huge. As you will see later on, this does not perform well on small graphs 

In [38]:
def build_trigram(sentence):
    
    '''
    Builds bigrams from list of words
    '''
    
    bigrams = []
    for i in range(2,len(sentence)):
        bigrams.append((sentence[i-2],sentence[i-1],sentence[i]))
    return bigrams

# Build bigrams

In [39]:
def build_bigram(sentence):
    
    '''
    Builds bigrams from list of words
    '''
    
    bigrams = []
    for i in range(1,len(sentence)):
        bigrams.append((sentence[i-1],sentence[i]))
    return bigrams

# Clearing punctuation and specific Regexing

In [40]:
import string
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 
def clean_punctuation_and_stopwords(question):
    
    '''
    Cleaning data for use for bigram usage
    '''

    question = re.sub(r'[^a-zA-Z0-9#+-]',' ',question.lower()).split()
    for i in range(len(question)):
        if question[i]=="c":
            question[i] = "C"
        elif question[i]=="r":
            question[i] = "R"
    question = list(filter(lambda x:x not in stop_words,question))
    for i in range(len(question)):
        if question[i] not in tags:
            question[i] = lemmatizer.lemmatize(question[i])
    return question

# Builds data from dataset, this function replaces global variables

In [41]:
def data_builder(fname):

    global d1,user_map
    d1 = {}
    user_map = {}
    fp = open(fname,"r")
    x = fp.readline()
    cnt = 0
    while True:
        if not cnt%10000:
            print(cnt,"Users read")
        cnt+=1
        x = fp.readline()
        uid = x.split(",")[0]
        uname = x.split(",")[1]
        about = ",".join(x.split(",")[2::])
        d1[uid] = about
        user_map[uid] = uname
        try:
            if uid=="12462789" or uid=="25000019":
                break
        except:
            pass

data_builder("parsed_data.csv")

0 Users read
10000 Users read
20000 Users read
30000 Users read
40000 Users read
50000 Users read
60000 Users read


# Calls jaccard and punctuation cleaner

In [42]:
def similar_questions(question1,question2):
    
    '''
    Checks how similar two questions are
    '''
    
    txt1 = clean_punctuation_and_stopwords(question1)
    txt2 = clean_punctuation_and_stopwords(question2)
    similarity = jaccard_similarity(txt1,txt2)
    return similarity

# Sample a question


In [43]:
def check_about(question,small=False):
    max_jacc = 0
    profile = None
    itr = 0
    usr_arr = []
    for i in d1:
        if not itr%10000:
            print(itr)
        itr+=1

        jacc = similar_questions(d1[i],question)
        if small:
            usr_arr.append((jacc,i))
        else:
            if jacc > 0:
                usr_arr.append((jacc,i))
            
    return usr_arr

# Prints working data


In [44]:
def full_data(usr_arr):
    full_str = []
    for i in sorted(usr_arr,reverse=True):
        full_str.extend(d1[i[1]].split())
        print(i[0],user_map[i[1]]+" : "+d1[i[1]])


    arr = list(filter(lambda x:x in tags,full_str))
    from collections import Counter
    print(Counter(arr).most_common())

# Recommended users are written in order. 

## Recommended programming languages at the end of output

In [50]:
usr_arr = check_about("I enjoy ruby and php")
full_data(usr_arr)

0
10000
20000
30000
40000
50000
60000
0.3333333333333333 Lilli Lieberenz : ruby, php, javascript

0.3333333333333333 rfei : ruby php ubuntu

0.25 Eason : Know a little ruby and php

0.16666666666666666 Willem : Developer python, ruby, php, java and c++.

0.14285714285714285 ruby007 : want to be a web developer. learning ruby and php now a days

0.1111111111111111 ZyDevs : Young programming enthusiast... Mainly do java or C# but can do some VB, ruby and php.

0.09090909090909091 mkoga : software engineer working for an internet startup. what a cliche, i know. my favorite languages are ruby and php.

0.08333333333333333 OpenThread : Senior Cocoa developer with since 2011. Across iOS and OSX.Junior ruby and php developer.

0.08333333333333333 Craig Webster : Clinical Biochemist by trade, interested in scientific computing and a rank amateur at programming, objective-c, ruby and php

0.06666666666666667 Stray : Girl geek with an obsession with TDD and all things symmetrical. Mostly buildin

# Builds Language dependency matrix. 

## This could take a while

In [14]:
matrix = {i:{j:0 for j in tags} for i in tags}
fp = open("data.csv","r")
line = fp.readline()
cnt=0
while len(line)>0:
    if cnt%50000==0:
        print(cnt,"lines complete")
    line = fp.readline()
    language = re.sub(r'[^a-zA-Z0-9#+-]',' ',line.lower())
    language = language.split()
    for i in range(len(language)):
        if language[i]=="c":
            language[i] = "C"
        elif language[i]=="r":
            language[i] = "R"
    
    mappings = list(filter(lambda x:x in set(tags),language))
    for i in mappings:
        for j in mappings:
            if i!=j:
                matrix[i][j]+=1
    cnt+=1
    
fp.close()

# JavaScript, HTML, CSS p(JavaScript/HTML)


0 lines complete
50000 lines complete
100000 lines complete
150000 lines complete
200000 lines complete
250000 lines complete
300000 lines complete
350000 lines complete
400000 lines complete
450000 lines complete
500000 lines complete
550000 lines complete
600000 lines complete
650000 lines complete
700000 lines complete
750000 lines complete
800000 lines complete
850000 lines complete
900000 lines complete


# Converts matrix to percentage weight

In [15]:
for i in matrix:
    matsum = 0
    for j in matrix[i]:
        matsum+=matrix[i][j]
    for j in matrix[i]:
        try:
            matrix[i][j] = (matrix[i][j]/matsum)
        except:
            matrix[i][j] = 0

# Returns languages for input

In [16]:
def return_langs(fs):
    my_langs = list(filter(lambda x:x in tags,fs.lower().split()))
    for i in range(len(my_langs)):
        if my_langs[i]=="c":
            my_langs[i] = "C"
        elif my_langs[i]=="r":
            my_langs[i] = "R"
    print(my_langs)
    m1 = {i:0 for i in matrix.keys() if i not in my_langs}
    for i in my_langs:
        for j in matrix[i]:
            try:
                m1[j]+=matrix[i][j]
            except:
                pass
    orderedTp = [(i,m1[i]) for i in m1]
    sortsum  = sum([i[1] for i in orderedTp])
    for i in sorted(orderedTp,key = lambda x:x[1],reverse=True):
        print(i[0],(i[1]/sortsum)*100,sep="\t\t")

# Friends data set to build a graph

In [17]:
data_builder("friends.csv")

friend_graph = {i:{j:0 for j in user_map.keys()} for i in user_map.keys()}

for i in d1:
    chk = check_about(d1[i],True)
    full_data(chk)
    for j in chk:
        
        if i!=j[1]:
            friend_graph[i][j[1]] = j[0]
    
    

            

0 Users read
0
1.0 roh : Thriving to learn and adapt to newer technologies.Heading towards learning and understanding while tackling real world problems. Currently researching on Virtual Reality and Machine Learning. Well versed in Python, Javascript, C and Angular.

0.0625 shr : Has a diverse skill set in technologies and is interested in the latest trends around the globe. Has worked with cloud technologies and also in the field of IoT. Also interested in data science/analytics as well as machine learning. Well versed in Django, Python and Mongo.

0.04 sre : I am graduate Student at USC with a strong focus on Machine Learning and Data Science.Apart from this, I also have full stack software development experience in an agile environment gained during my 6 month internship at GE Digital. Well versed in Perl, Ruby and C++.

0.037037037037037035 And :  author of The Hundred-Page Machine Learning Book

0.025 dalia : Senior undergraduate at IIT Delhi passionate about the application of Co

1.0 rhy : Research Intern at Center for Cloud Computing and Big Data, PES University. Aatmatrisha is the annual techno-cultural fest of PES University. I volunteered for Debate in 2016. In 2017, I was a Web Developer for the official website, while being the organiser for Debate. Worked with oracle firebase and am familiar with typescript.

0.0784313725490196 ave : I'm a 4th year student at PES University, interested in Machine learning, Natural Language Processing, cloud computing and Big data.  Developed a software interface in Python, including GUI, based on a Raspberry Pi for a 3D Rock Surface Profiler.

0.05357142857142857 sur :  I look forward to accepting challenging work and contribute towards the success of esteemed organization by hard work and acquired skills. My fields of interest primarily include Cloud Computing and Big data. I have also worked on projects which include Data Analytics, Machine Learning, Computer Networks. 

0.03225806451612903 anm : A highly motivated stu

1.0 ave : I'm a 4th year student at PES University, interested in Machine learning, Natural Language Processing, cloud computing and Big data.  Developed a software interface in Python, including GUI, based on a Raspberry Pi for a 3D Rock Surface Profiler.

0.0784313725490196 rhy : Research Intern at Center for Cloud Computing and Big Data, PES University. Aatmatrisha is the annual techno-cultural fest of PES University. I volunteered for Debate in 2016. In 2017, I was a Web Developer for the official website, while being the organiser for Debate. Worked with oracle firebase and am familiar with typescript.

0.07407407407407407 sur :  I look forward to accepting challenging work and contribute towards the success of esteemed organization by hard work and acquired skills. My fields of interest primarily include Cloud Computing and Big data. I have also worked on projects which include Data Analytics, Machine Learning, Computer Networks. 

0.06779661016949153 anm : A highly motivated stu

1.0 lak : Working on the optimisation of cost of video transfer using ML based approaches to downscale and upscale videos before and after transfer to minimise quality loss. Subject Matter Expert (SME) for the course Smart Cities & IoT.  In addition, included concepts of IoT, sensors and Arduinos. Emulation done using the concept of containerisation specific to Windows Docker as compared to Linux docker containers.

0.031746031746031744 div : Computer Science and Engineering B.Tech student at PES University, Bangalore with a passion for art and design. Subject Matter Expert in Design and Adobe Photoshop as part of the PESU i/o program. Used C++, Java, Python and matlab

0.02112676056338028 sun : A highly motivated learner intrigued by all aspects of computer science, with a particular focus on data analysis driven by the thought of coming up with unique and weird insights that could surprise, effectively change perspectives and lead to positive development. As a side hobby I have loved

1.0 Guy : Over 20 years of intensive face to face customer service experienceKnowledgeable of Microsoft CRM, Excel, Microsoft Word, Salesforce. Knowledge of mortgage industry standard loan processing software Calyx Point, Encompass, and Byte Dedicated and loyal individual with strong leadership abilities. Strong ability to coordinate, manage and complete multiple tasks with changing priorities.

0.0 Aar :  I am a machine learning researcher and I worked on a project that Optimized neural networks to achieve significant reduction in processing time in predicting strain fields in micro-structure with high accuracy using Tensorflow and Keras libraries

0.0 Mad : A cadet by nature and a developer by passion, a tech enthusiast, a full-stack JavaScript developer (https://github.com/MadhavBahlMD), tech blogger (https://medium.com/@madhavbahl10), self-help blogger, writer, motivator and mentor for those who want to learn programming, development and recent technologies. Being an active part of

0.0 rhy : Research Intern at Center for Cloud Computing and Big Data, PES University. Aatmatrisha is the annual techno-cultural fest of PES University. I volunteered for Debate in 2016. In 2017, I was a Web Developer for the official website, while being the organiser for Debate. Worked with oracle firebase and am familiar with typescript.

0.0 ish :  SME for C Programming Language - Taught a class of 40 students the basics of programming, introduction to Algorithms and Data Structures, sequential, selective and iteration programming paradigms, their implementation in C, Arrays and Strings in C and concept of pointers over 4 weeks. Coordinator for Microsoft Code.fun.do++ at PES University. Responsibilities included marketing the event, Microsoft Azure and Github, conduct seminars on Azure and ML for the participants and coordinate the deadlines and submissions with the teams and Microsoft ACAD. Also hosted the University Relations Director of Microsoft from Redmond, WA.

[('C', 6), ('a

# Structure of friend graph

In [18]:
print(friend_graph)

{'25000002': {'25000002': 0, '25000003': 0.0, '25000004': 0.018518518518518517, '25000005': 0.0625, '25000006': 0.04, '24000001': 0.0, '24000002': 0.0, '24000003': 0.0, '24000004': 0.0, '24000005': 0.015625, '24000006': 0.0, '24000007': 0.0, '24000008': 0.01020408163265306, '24000009': 0.020833333333333332, '24000010': 0.013157894736842105, '24000011': 0.02, '24000012': 0.0, '24000013': 0.0, '24000014': 0.015625, '24000015': 0.0, '24000016': 0.014084507042253521, '24000017': 0.0, '24000018': 0.01694915254237288, '24000019': 0.011627906976744186, '24000020': 0.0, '24000021': 0.0, '25000007': 0.0, '25000008': 0.022727272727272728, '26000001': 0.037037037037037035, '26000002': 0.0, '26000003': 0.0, '26000004': 0.0, '26000005': 0.0, '26000006': 0.014705882352941176, '26000007': 0.022222222222222223, '25000010': 0.0, '25000011': 0.0, '25000012': 0.0, '25000013': 0.024390243902439025, '25000014': 0.023255813953488372, '25000015': 0.0, '25000016': 0.0, '25000017': 0.0, '25000018': 0.0, '25000

# Person v Person comparison

In [19]:
for i in friend_graph:
    for j in friend_graph[i]:
        print(user_map[i],user_map[j],friend_graph[i][j],sep="\t\t")

roh		roh		0
roh		pri		0.0
roh		sur		0.018518518518518517
roh		shr		0.0625
roh		sre		0.04
roh		vai		0.0
roh		nish		0.0
roh		ish		0.0
roh		rhy		0.0
roh		rrt		0.015625
roh		sun		0.0
roh		mid		0.0
roh		nav		0.01020408163265306
roh		apa		0.020833333333333332
roh		aay		0.013157894736842105
roh		ave		0.02
roh		nee		0.0
roh		nih		0.0
roh		dwe		0.015625
roh		sha		0.0
roh		kau		0.014084507042253521
roh		lak		0.0
roh		anm		0.01694915254237288
roh		sey		0.011627906976744186
roh		div		0.0
roh		adi		0.0
roh		Cha		0.0
roh		Meg		0.022727272727272728
roh		And		0.037037037037037035
roh		kav		0.0
roh		Sumi		0.0
roh		Guy		0.0
roh		Brig		0.0
roh		Mad		0.014705882352941176
roh		Aar		0.022222222222222223
roh		rish		0.0
roh		man		0.0
roh		kay		0.0
roh		shal		0.024390243902439025
roh		mic		0.023255813953488372
roh		mik		0.0
roh		ram		0.0
roh		shar		0.0
roh		roy		0.0
roh		dalia		0.025
pri		roh		0.0
pri		pri		0
pri		sur		0.024390243902439025
pri		shr		0.02702702702702703
pri		sre		0.0
pri		vai		0.0
pri		nish		0.

rrt		sun		0.006896551724137931
rrt		mid		0.0
rrt		nav		0.02654867256637168
rrt		apa		0.047619047619047616
rrt		aay		0.044444444444444446
rrt		ave		0.014925373134328358
rrt		nee		0.01639344262295082
rrt		nih		0.0
rrt		dwe		0.012345679012345678
rrt		sha		0.0375
rrt		kau		0.011363636363636364
rrt		lak		0.0
rrt		anm		0.02666666666666667
rrt		sey		0.009708737864077669
rrt		div		0.03125
rrt		adi		0.0
rrt		Cha		0.0
rrt		Meg		0.05084745762711865
rrt		And		0.022727272727272728
rrt		kav		0.0
rrt		Sumi		0.0
rrt		Guy		0.0
rrt		Brig		0.0
rrt		Mad		0.0
rrt		Aar		0.016129032258064516
rrt		rish		0.0
rrt		man		0.0
rrt		kay		0.0
rrt		shal		0.03508771929824561
rrt		mic		0.03389830508474576
rrt		mik		0.0
rrt		ram		0.014285714285714285
rrt		shar		0.016129032258064516
rrt		roy		0.016129032258064516
rrt		dalia		0.03571428571428571
sun		roh		0.0
sun		pri		0.0
sun		sur		0.007407407407407408
sun		shr		0.0
sun		sre		0.0
sun		vai		0.014492753623188406
sun		nish		0.006666666666666667
sun		ish		0.006060606060606061

kau		ave		0.013513513513513514
kau		nee		0.0
kau		nih		0.0
kau		dwe		0.011363636363636364
kau		sha		0.0
kau		kau		0
kau		lak		0.0
kau		anm		0.012048192771084338
kau		sey		0.00909090909090909
kau		div		0.0
kau		adi		0.0
kau		Cha		0.0
kau		Meg		0.014705882352941176
kau		And		0.0196078431372549
kau		kav		0.0
kau		Sumi		0.0
kau		Guy		0.0
kau		Brig		0.0
kau		Mad		0.0
kau		Aar		0.014492753623188406
kau		rish		0.0
kau		man		0.0
kau		kay		0.0
kau		shal		0.015384615384615385
kau		mic		0.014925373134328358
kau		mik		0.0
kau		ram		0.0
kau		shar		0.0
kau		roy		0.0
kau		dalia		0.015625
lak		roh		0.0
lak		pri		0.0
lak		sur		0.0
lak		shr		0.0
lak		sre		0.0
lak		vai		0.0
lak		nish		0.0
lak		ish		0.0
lak		rhy		0.0
lak		rrt		0.0
lak		sun		0.02112676056338028
lak		mid		0.0
lak		nav		0.008771929824561403
lak		apa		0.0
lak		aay		0.0
lak		ave		0.0
lak		nee		0.0
lak		nih		0.0
lak		dwe		0.0
lak		sha		0.0
lak		kau		0.0
lak		lak		0
lak		anm		0.0
lak		sey		0.0
lak		div		0.031746031746031744
lak		adi		0.0
lak		Ch

Guy		kau		0.0
Guy		lak		0.0
Guy		anm		0.0
Guy		sey		0.0
Guy		div		0.0
Guy		adi		0.0
Guy		Cha		0.0
Guy		Meg		0.0
Guy		And		0.0
Guy		kav		0.0
Guy		Sumi		0.0
Guy		Guy		0
Guy		Brig		0.0
Guy		Mad		0.0
Guy		Aar		0.0
Guy		rish		0.0
Guy		man		0.0
Guy		kay		0.0
Guy		shal		0.0
Guy		mic		0.0
Guy		mik		0.0
Guy		ram		0.0
Guy		shar		0.0
Guy		roy		0.0
Guy		dalia		0.0
Brig		roh		0.0
Brig		pri		0.0
Brig		sur		0.0
Brig		shr		0.0
Brig		sre		0.0
Brig		vai		0.0
Brig		nish		0.0
Brig		ish		0.0
Brig		rhy		0.0
Brig		rrt		0.0
Brig		sun		0.0
Brig		mid		0.0
Brig		nav		0.0
Brig		apa		0.0
Brig		aay		0.0
Brig		ave		0.0
Brig		nee		0.0
Brig		nih		0.0
Brig		dwe		0.0
Brig		sha		0.0
Brig		kau		0.0
Brig		lak		0.0
Brig		anm		0.0
Brig		sey		0.0
Brig		div		0.0
Brig		adi		0.0
Brig		Cha		0.0
Brig		Meg		0.0
Brig		And		0.0
Brig		kav		0.0
Brig		Sumi		0.0
Brig		Guy		0.0
Brig		Brig		0
Brig		Mad		0.0
Brig		Aar		0.0
Brig		rish		0.0
Brig		man		0.0
Brig		kay		0.0
Brig		shal		0.022222222222222223
Brig		mic		0.0
Brig		mik		0.0
Brig		ram	

roy		sre		0.0
roy		vai		0.0
roy		nish		0.0
roy		ish		0.0
roy		rhy		0.02040816326530612
roy		rrt		0.016129032258064516
roy		sun		0.007936507936507936
roy		mid		0.0
roy		nav		0.010416666666666666
roy		apa		0.0
roy		aay		0.0
roy		ave		0.020833333333333332
roy		nee		0.0
roy		nih		0.0
roy		dwe		0.016129032258064516
roy		sha		0.0
roy		kau		0.0
roy		lak		0.0
roy		anm		0.0
roy		sey		0.0
roy		div		0.0
roy		adi		0.0
roy		Cha		0.023809523809523808
roy		Meg		0.0
roy		And		0.0
roy		kav		0.0
roy		Sumi		0.0
roy		Guy		0.0
roy		Brig		0.0
roy		Mad		0.0
roy		Aar		0.023255813953488372
roy		rish		0.0
roy		man		0.0
roy		kay		0.0
roy		shal		0.0
roy		mic		0.0
roy		mik		0.0
roy		ram		0.0
roy		shar		1.0
roy		roy		0
roy		dalia		0.0
dalia		roh		0.025
dalia		pri		0.0
dalia		sur		0.02127659574468085
dalia		shr		0.023255813953488372
dalia		sre		0.022727272727272728
dalia		vai		0.0196078431372549
dalia		nish		0.016129032258064516
dalia		ish		0.0
dalia		rhy		0.0
dalia		rrt		0.03571428571428571
dalia		sun		0.0082644628

# Writes Edge graph for friend's matrix using Gephi format

In [20]:
'''
;A;B;C;D;E
A;0;1;0;1;0
B;1;0;0;0;0
C;0;0;1;0;0
D;0;1;0;1;0
E;0;0;0;0;0
'''
edg_list = open("edge.csv","w")
fkeys = ";"+";".join([user_map[i] for i in friend_graph.keys()])
edg_list.write(fkeys+"\n")
for i in friend_graph:
    edg_list.write(user_map[i])
    for j in friend_graph[i]:
        edg_list.write(";"+str(friend_graph[i][j]))
    edg_list.write("\n")
edg_list.close()

# Programming language dependency matrix to Gephi format

In [21]:
edg_list = open("progmatrix.csv","w")
fkeys = ";"+";".join([i for i in matrix.keys()])
edg_list.write(fkeys+"\n")
for i in matrix:
    edg_list.write(i)
    for j in matrix[i]:
        edg_list.write(";"+str(matrix[i][j]))
    edg_list.write("\n")
edg_list.close()

# Sample from language dependency matrix

## C++ naturally suggests C language

In [22]:
return_langs("I love c++")

['c++']
C		12.859446884731732
java		12.61269296231057
python		9.341717020818004
c#		7.974908396322476
javascript		5.847027432793003
php		5.364668108541998
net		4.07961530171613
html		3.8878607475454676
linux		3.450095505659733
android		3.267259767962125
sql		3.129018112629788
css		2.9015882925669105
ruby		1.5020773410035158
asp		1.4983611674730768
jquery		1.4418753298104012
ios		1.3682950939077054
perl		1.3437683486068068
matlab		0.958772770853308
R		0.9060031067210715
swift		0.8294499319940246
objective-c		0.8227608196392341
go		0.8026934825748625
bash		0.7722208596252613
unix		0.7714776249191734
node		0.7670182166826464
xml		0.7157350219625858
delphi		0.6116821631102888
git		0.5455342742684713
angular		0.5217507636736606
shell		0.49573754896058636
rails		0.46972433424751214
unity		0.46675139542316074
react		0.4652649260109851
scala		0.4318193642370325
django		0.420670843645715
spring		0.41546820070310014
mongodb		0.3954008636387286
haskell		0.3901982206961137
bootstrap		0.37904970010

## Cocoa and Swift are programming languages for Apple app development. 

## Objective-C is by Apple as well

In [23]:
return_langs("I love cocoa and swift")

['cocoa', 'swift']
ios		20.19933902814618
objective-c		7.981451985764442
C		6.860615495585108
java		6.43815962596672
android		5.356668001301781
c++		4.268624149675835
javascript		4.111279507550652
php		3.9881573319964105
python		3.667740445215726
c#		3.4030547946740173
net		2.982878248884156
html		2.5102424351057753
css		2.2967708254464583
ruby		2.1502927657152298
sql		1.959764728017557
json		1.796489643438421
xml		1.4148200739961152
git		1.4062145764184797
jquery		1.3896187334021883
linux		1.121446668042257
react		1.090095464150556
rails		1.0784241544301367
node		1.0042582511260063
asp		0.9311247864522566
firebase		0.7453084935747084
angular		0.7407995886782186
perl		0.5588803647422189
go		0.5435131679118672
unity		0.5174954933473453
mongodb		0.5160606910537332
django		0.4716050332400196
wordpress		0.46443433795599637
R		0.3843313598298392
unix		0.3828982156282459
spring		0.3826920595205735
laravel		0.37101411743207924
bootstrap		0.34212849637235176
bash		0.33434430704136814
delphi		0

## Gradle and Maven are built for Java, and Spring uses both Gradle and Maven for package dependency. Android pairs with Java, as does Hibernate

In [24]:
return_langs("I love gradle and maven")

['gradle', 'maven']
java		16.253541029350643
spring		10.69253285937903
android		5.219207707456439
hibernate		4.252913706332635
git		4.163974224108209
sql		3.9568021123143255
javascript		3.937601261840673
apache		3.1375907166322965
docker		2.595781936129478
linux		2.3839573355414205
html		2.2789435532332862
tomcat		2.274291064439112
jquery		2.264690639202286
css		2.2259934906065015
python		2.191948830804891
selenium		2.0491242428464087
xml		1.9490827470618464
mongodb		1.8595280194595456
jpa		1.8266651502518494
angular		1.4990445515249173
C		1.4792528057543073
php		1.4636460272242346
json		1.383396277210282
c++		1.3148394360534243
net		1.1109659485546424
c#		1.0098660359194862
ajax		1.0023580943029287
react		0.9710211634994624
bootstrap		0.9143049550239394
node		0.8946609330775688
ruby		0.7748033414414773
shell		0.7734494769424051
bash		0.7220010026389319
scala		0.7044494643128298
ios		0.6340955871841822
unix		0.6184888086541096
ubuntu		0.5976386460329058
typescript		0.5350881581692947
h